In [1]:
import ray
from ray import serve
from starlette.requests import Request
from fastapi import FastAPI, File, HTTPException, Security, Depends
from fastapi.security import HTTPAuthorizationCredentials, HTTPBearer
from faster_whisper import WhisperModel
import os
import io
import logging
import time
from logging.handlers import RotatingFileHandler

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-18 11:56:51,419	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
model = WhisperModel(
            'large-v3',
            device='cuda',
            compute_type='float16',
        )

In [3]:
initial_prompt = 'pay attention on the following words: vexa'

In [5]:
initial_prompt = None
prefix = None

In [41]:
prefix = 'привет,'

In [42]:
result,_ = model.transcribe('app/test_audio.webm',initial_prompt=initial_prompt,prefix=prefix,task='transcribe',language='ru')

In [43]:
_

TranscriptionInfo(language='ru', language_probability=1, duration=314.88, duration_after_vad=314.88, all_language_probs=None, transcription_options=TranscriptionOptions(beam_size=5, best_of=5, patience=1, length_penalty=1, repetition_penalty=1, no_repeat_ngram_size=0, log_prob_threshold=-1.0, no_speech_threshold=0.6, compression_ratio_threshold=2.4, condition_on_previous_text=True, prompt_reset_on_temperature=0.5, temperatures=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0], initial_prompt=None, prefix='привет,', suppress_blank=True, suppress_tokens=[-1], without_timestamps=False, max_initial_timestamp=1.0, word_timestamps=False, prepend_punctuations='"\'“¿([{-', append_punctuations='"\'.。,，!！?？:：”)]}、'), vad_options=None)

In [44]:
transcribe_list = list(result)


In [45]:
transcribe_list

[Segment(id=1, seek=2294, start=0.0, end=10.32, text=' привет, Роза, вы смогли прийти, добро пожаловать, это хорошо, спасибо за то, что вы здесь,', tokens=[50365, 33879, 11, 6325, 3434, 386, 11, 2840, 32139, 1675, 5082, 1644, 3396, 11, 16991, 9938, 713, 18327, 14069, 11, 2691, 16977, 11, 37536, 4396, 4572, 11, 2143, 2840, 9087, 11, 50881], temperature=0.0, avg_logprob=-0.4306469340073435, compression_ratio=1.5914634146341464, no_speech_prob=0.1939697265625, words=None),
 Segment(id=2, seek=2294, start=10.32, end=15.92, text=' так что это Санкт-Петербург,', tokens=[50881, 2936, 2143, 2691, 2933, 1416, 42964, 12, 9223, 1094, 41934, 7489, 1906, 11, 51161], temperature=0.0, avg_logprob=-0.4306469340073435, compression_ratio=1.5914634146341464, no_speech_prob=0.1939697265625, words=None),
 Segment(id=3, seek=2294, start=15.92, end=22.94, text=' давайте посмотрим, кто еще,', tokens=[51161, 30412, 42293, 11, 12278, 9910, 11, 51512], temperature=0.0, avg_logprob=-0.4306469340073435, compressio

In [28]:
Segment

NameError: name 'Segment' is not defined

In [2]:

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"


app = FastAPI()
security = HTTPBearer()

# Get configurations from environment variables
API_TOKEN = os.getenv("WHISPER_API_TOKEN", "default_token_change_me")
MODEL_SIZE = os.getenv("WHISPER_MODEL_SIZE", "large-v3")
NUM_CPUS = float(os.getenv("WHISPER_NUM_CPUS", "1"))
NUM_GPUS = float(os.getenv("WHISPER_NUM_GPUS", "0.1"))
DEVICE = os.getenv("WHISPER_DEVICE", "cuda")
COMPUTE_TYPE = os.getenv("WHISPER_COMPUTE_TYPE", "float16")
BEAM_SIZE = int(os.getenv("WHISPER_BEAM_SIZE", "5"))
VAD_FILTER = os.getenv("WHISPER_VAD_FILTER", "True").lower() == "true"
VAD_THRESHOLD = float(os.getenv("WHISPER_VAD_THRESHOLD", "0.9"))
NUM_REPLICAS = os.getenv("NUM_REPLICAS", "1")

def verify_token(credentials: HTTPAuthorizationCredentials = Security(security)) -> bool:
    if credentials.credentials != API_TOKEN:
        raise HTTPException(
            status_code=401,
            detail="Invalid authentication token",
            headers={"WWW-Authenticate": "Bearer"},
        )
    return True

@serve.deployment(num_replicas=2, ray_actor_options={"num_cpus": 0, "num_gpus": 1})
class Transcriber:
    def __init__(self):
        # Set up logging
        self.logger = logging.getLogger("transcriber")
        self.logger.setLevel(logging.INFO)
        
        # Clear any existing handlers
        self.logger.handlers = []
        
        # Create logs directory if it doesn't exist
        log_directory = os.path.join(os.getcwd(), 'logs')
        os.makedirs(log_directory, exist_ok=True)
        log_file = os.path.join(log_directory, 'transcriber.log')
        
        # Create a file handler with proper permissions
        file_handler = RotatingFileHandler(
            log_file,
            maxBytes=10*1024*1024,  # 10MB
            backupCount=5,
            mode='a'
        )
        formatter = logging.Formatter('[%(asctime)s] %(levelname)s in %(module)s: %(message)s')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        
        # Also add a stream handler for console output
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)
        
        self.logger.info("Initializing Transcriber service")
        
        a
        self.logger.info(f"Initialized Whisper model: {MODEL_SIZE} on {DEVICE} with {COMPUTE_TYPE}")

    def transcribe(self, audio_bytes: bytes, request_id: str) -> dict:
        self.logger.info(f"[{request_id}] Starting transcription")
        transcribe_start = time.time()
        segments, info = self.model.transcribe(
            io.BytesIO(audio_bytes),
            beam_size=BEAM_SIZE,
            vad_filter=VAD_FILTER,
            word_timestamps=True,
            vad_parameters={"threshold": VAD_THRESHOLD},
        )
        transcribe_end = time.time()
        transcribe_duration = transcribe_end - transcribe_start
        
        return {
            "segments": segments,
            "timing": {
                "transcribe_time_ms": transcribe_duration * 1000,
                "realtime_factor": transcribe_duration/info.duration if info.duration else 0
            }
        }

    async def __call__(self, request: Request) -> dict:
        # Verify authentication
        auth_header = request.headers.get("Authorization")
        if not auth_header or auth_header.split()[1] != API_TOKEN:
            raise HTTPException(
                status_code=401,
                detail="Invalid authentication token",
                headers={"WWW-Authenticate": "Bearer"},
            )

        request_id = str(time.time())
        start_time = time.time()
        
        try:
            # Read raw bytes from request body
            self.logger.info(f"[{request_id}] Starting request processing")
            audio_bytes = await request.body()
            body_read_time = time.time()
            audio_size = len(audio_bytes) / 1024  # Size in KB
            
            self.logger.info(
                f"[{request_id}] Read request body: {audio_size:.2f}KB in "
                f"{(body_read_time - start_time)*1000:.2f}ms"
            )

            # Get transcription
            result = self.transcribe(audio_bytes, request_id)
            
            # Add timing information
            result["timing"]["request_id"] = request_id
            result["timing"]["body_read_time_ms"] = (body_read_time - start_time) * 1000
            result["timing"]["total_time_ms"] = (time.time() - start_time) * 1000
            
            self.logger.info(
                f"[{request_id}] Transcription completed:\n"
                f"  - Model inference time: {result['timing']['transcribe_time_ms']:.2f}ms\n"
                f"  - Total request time: {result['timing']['total_time_ms']:.2f}ms\n"
                f"  - Audio/Processing ratio: {result['timing']['realtime_factor']:.2f}x realtime"
            )
            
            return result
            
        except Exception as e:
            error_time = time.time()
            self.logger.error(
                f"[{request_id}] Error during transcription after "
                f"{(error_time - start_time)*1000:.2f}ms: {str(e)}"
            )
            raise HTTPException(status_code=500, detail=str(e))

transcriber_app = Transcriber.bind()